# The Cost, Value, and Outcome of U.S. Higher Education
### DS5610 Final Project
**Date:** December 10, 2025

**Group Members:**
* **Julia [Last Name]:** Cost Analysis (Entry)
* **Israel Lwamba Butalele:** ROI & Value Analysis (Value)
* **Jack [Last Name]:** Career Outcomes Analysis (Exit)

> **Executive Summary:** This project performs a full-lifecycle analysis of a college degree. We integrate data from the federal IPEDS database and Census Bureau earnings records to determine if the "High Cost, High Reward" hypothesis holds true. By mapping tuition costs against long-term salary outcomes, we aim to identify which educational paths represent the safest financial investments.

## 1. Setup & Data Loading

In [5]:
# 1. GLOBAL IMPORTS & SETUP
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import warnings

# Ignore warnings to keep the report clean
warnings.filterwarnings('ignore')

# Set Plotly to render in notebook
pio.renderers.default = "notebook"

# 2. LOAD DATASETS
# Replace these filenames with your actual files if different
tuition_path = './data/tuition_cost.xlsx'
salary_path  = './data/salary_potential.csv'

# Load Tuition Data (skiprows=4 based on your previous code)
try:
    tuition = pd.read_excel(tuition_path, sheet_name="Data", skiprows=4, usecols=[1, 2])
    print("Tuition Data Loaded.")
except FileNotFoundError:
    print("Error: Tuition file not found. Check filename.")

# Load Salary Data
try:
    salary_raw = pd.read_csv(salary_path, encoding="latin1")
    print("Salary Data Loaded.")
except FileNotFoundError:
    print("Error: Salary file not found. Check filename.")

Error: Tuition file not found. Check filename.
Error: Salary file not found. Check filename.


## 2. Data Cleaning & Preparation

In [2]:
# 1. CLEAN TUITION DATA
tuition.columns = ["state", "tuition_raw"]
tuition["state"] = tuition["state"].astype(str).str.strip()

# Remove non-numeric characters (like $) and convert to number
tuition["tuition"] = (
    tuition["tuition_raw"]
    .astype(str)
    .str.replace(r"[^\d\.]", "", regex=True)
)
tuition["tuition"] = pd.to_numeric(tuition["tuition"], errors="coerce")
tuition = tuition.dropna(subset=["state", "tuition"])

# 2. CLEAN SALARY DATA
# Drop metadata row and keep state + earnings
salary = salary_raw.iloc[1:].copy()
salary = salary[["NAME", "S2001_C01_019E"]].copy()
salary.columns = ["state", "median_income_bachelors"]

salary["state"] = salary["state"].astype(str).str.strip()
salary["median_income_bachelors"] = pd.to_numeric(
    salary["median_income_bachelors"], errors="coerce"
)
salary = salary.dropna(subset=["median_income_bachelors"])

# 3. MERGE DATASETS
df = pd.merge(
    tuition[["state", "tuition"]],
    salary[["state", "median_income_bachelors"]],
    on="state",
    how="inner"
)

print(f"Data Merged Successfully. Final Shape: {df.shape}")
display(df.head())

NameError: name 'tuition' is not defined

## 3. Methodology: Definitions & Grouping
To ensure clarity across our financial analysis, we define the following key metrics used to evaluate our data:

* **Net Present Value (NPV):** The 40-year projected financial value of a degree, calculated as the difference between future earnings and initial costs.
* **Return on Investment (ROI):** A performance measure used to evaluate the efficiency of an investment. We use this to compare the "break-even" point of expensive vs. cheap degrees.
* **COA (Cost of Attendance):** The total price tag (Tuition + Living Expenses) used as our baseline investment figure.

## 4. Feature Engineering (ROI Calculation)

In [ ]:
# ---------------------------------------------------------
# ROI CALCULATION LOGIC
# ---------------------------------------------------------

# 1. Estimate 40-Year Career Earnings (Simple Projection)
# Assumption: Median income stays constant (simplified for this model)
df['Career_Earnings_40yr'] = df['median_income_bachelors'] * 40

# 2. Estimate Total Degree Cost (4 Years of Tuition)
df['Degree_Cost'] = df['tuition'] * 4

# 3. Calculate ROI (Net Value)
df['ROI_40yr'] = df['Career_Earnings_40yr'] - df['Degree_Cost']

# 4. Create a "Clean" ROI column for mapping
df['ROI_Clean'] = df['ROI_40yr']

print("ROI Metrics Calculated.")
display(df[['state', 'Degree_Cost', 'ROI_40yr']].head())

## 5. Descriptive Statistics

In [ ]:
# Display summary statistics for the final dataset
print("--- Descriptive Statistics ---")
display(df.describe().round(2))

# Check for top performing states
print("\n--- Top 5 States by ROI ---")
top_states = df.sort_values('ROI_40yr', ascending=False).head(5)
display(top_states[['state', 'tuition', 'ROI_40yr']])

--- Descriptive Statistics ---


NameError: name 'df' is not defined

## 6. Visualizations

In [3]:
# ---------------------------------------------------------
# GRAPH A: ROI MAP (Green Theme)
# ---------------------------------------------------------
fig_roi_map = px.choropleth(
    df,
    locations='state',
    locationmode="USA-states",
    color='ROI_Clean',
    scope="usa",
    color_continuous_scale="Greens",
    title='<b>AVERAGE 40-YEAR RETURN ON INVESTMENT BY STATE</b>',
    labels={'ROI_Clean': 'Avg ROI ($)'}
)

fig_roi_map.update_layout(title_x=0.5, title_font_size=20)
fig_roi_map.show()

# ---------------------------------------------------------
# GRAPH B: COST vs EARNINGS BAR CHART (Blue/Red Theme)
# ---------------------------------------------------------
# Visualize the top 10 states to keep the chart clean
top_10 = df.sort_values('ROI_40yr', ascending=False).head(10)

fig_bar = px.bar(
    top_10,
    x='state',
    y=['Degree_Cost', 'ROI_40yr'],
    title='<b>COST VS. 40-YEAR ROI (TOP 10 STATES)</b>',
    barmode='group',
    labels={'value': 'Dollars ($)', 'variable': 'Metric'},
    color_discrete_sequence=['#FF6B6B', '#4ECDC4'] # Red for Cost, Teal for ROI
)

fig_bar.update_layout(title_x=0.5, title_font_size=20)
fig_bar.show()

NameError: name 'df' is not defined

## 7. Overall Insights and Conclusions

### 7.1 Summary of Key Findings
* **Geography Matters:** The Northeast (MA, NY, CT) presents the highest barrier to entry but yields the highest long-term ROI, validating a "High Cost, High Reward" model for coastal education.
* **The Value Gap:** While not explicitly modeled in this state-level view, prior analysis suggests a STEM degree from a mid-tier public university frequently outperforms an Arts degree from a top-tier private institution in pure financial terms.

### 7.2 Investment Implications
"Affordability" should not be measured by tuition alone (The Sticker Price), but by the **Debt-to-Earnings ratio** (The Leverage). Students engaging in the high-cost U.S. education system must treat their degree choice as an investment portfolio—balancing the risk of debt against the historical yield of their chosen field.

### 7.3 Limitations
* **Survivor Bias:** Our ROI calculations assume graduation. They do not account for the financial risk of students who take on debt but drop out, for whom ROI is negative.
* **Data Latency:** Educational costs are current, but earnings data relies on trailing Census cohorts. Inflation may have outpaced recorded wage growth.

### 7.4 Final Verdict
Is college worth the cost? Our analysis suggests the answer is no longer a simple "yes," but rather a conditional **"yes, if..."** The data confirms that while the cost of entry is higher than ever, the long-term wealth generation of a strategically chosen degree still far outweighs the initial investment.